## Age Phase 3 (Data Preparation): Pclass

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 12](https://data-science.training/kapitel-12/)
* Datum: 23.03.2023

Wir betrachten das ordinalskalierte Attribut Pclass und erzeugen hierzu Dummy-Variablen (mittels One Hot Encoding). Diese Features analysieren wir dann in Bezug auf lineare Korrelationen zum Attribut Age.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd
# NumPy Paket (Package) importieren
#  Mehrdimensionale Datenstrukturen (Vektoren, Matrizen, Tensoren, Arrays), Lineare Algebra
#  https://numpy.org/doc/
import numpy as np

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Trainings- und Testdaten zusammenführen
#  (KNIME "Concatenate")
df = pd.concat([df_train, df_test], ignore_index=True)

In [7]:
# Datentypen automatisch konvertieren
df = df.convert_dtypes()

In [8]:
# Fehlende Werte behandeln (d.h. schätzen)
#  (KNIME: "Missing Values")
# Fare (Kardinalskala): 1 fehlender Wert => Benutze den konstanten Wert 7,896
#df['Fare'] = df['Fare'].fillna(7.896)
display(df[df['Fare'].isna()])
df.loc[1043, 'Fare'] = 7.896
display(df[df['PassengerId'] == 1044])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,<NA>,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,<NA>,<NA>,S


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,<NA>,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,7.896,<NA>,S


In [9]:
# One Hot Encoding => Dummy-Variablen
#  (KNIME: "One To Many")
df = pd.get_dummies(df, columns=['Pclass'], dtype=int)
# Ergebnis anzeigen
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   Int64  
 1   Survived     891 non-null    Int64  
 2   Name         1309 non-null   string 
 3   Sex          1309 non-null   string 
 4   Age          1046 non-null   Float64
 5   SibSp        1309 non-null   Int64  
 6   Parch        1309 non-null   Int64  
 7   Ticket       1309 non-null   string 
 8   Fare         1309 non-null   Float64
 9   Cabin        295 non-null    string 
 10  Embarked     1307 non-null   string 
 11  Pclass_1     1309 non-null   int32  
 12  Pclass_2     1309 non-null   int32  
 13  Pclass_3     1309 non-null   int32  
dtypes: Float64(2), Int64(4), int32(3), string(5)
memory usage: 135.6 KB


In [10]:
# Korrelationsmatrix mit den linearen Korrelationskoeffizienten nach Pearson
#  (KNIME: "Linear Correlation")
def dst_correlation_matrix(df):
    # Nur numerische Attribute auswählen
    df1 = df.select_dtypes(include=[np.number])
    # Korrelationsmatrix berechnen
    corr_matrix = df1.corr(method='pearson')
    # Rückgabe
    return corr_matrix

In [11]:
# Korrelationsmatrix berechnen und anzeigen
corr_matrix = dst_correlation_matrix(df)['Age'].sort_values()
display(corr_matrix)

Pclass_3      -0.337069
SibSp         -0.243699
Parch         -0.150917
Survived      -0.077221
Pclass_2      -0.014986
PassengerId    0.028814
Fare           0.177280
Pclass_1       0.393198
Age            1.000000
Name: Age, dtype: float64

### Ergebnisse der Korrelationsanalyse der neuen Features zum Attribut Age

| Attribut           | R      |
|--------------------|--------|
| Pclass_1           | +0,393 |
| Pclass_2           | -0,015 |
| Pclass_3           | -0,337 |

Passagiere der ersten und dritten Klasse zeigen eine schwache positive bzw. negative lineare Korrelation zum Alter.